# Digit Recognizer
This is a simple image classification challenge using the famous [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database). Lets see if we can train a classifier that can accurately distinguish handwritten digits. For my implementation, instead of using a traditional machine learning classifier like Support Vector Machine or Random Forest, I will be using a [Convolutional Neural Network (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network), as they are by far the best choice for image data.

<img src="images/MnistExamples.png">

### Goal
The goal is to take an image of a handwritten single digit, and determine what that digit is. For every image in the test set, we want to predict the correct label.

### Metric
This competition is evaluated on the categorization accuracy of our predictions (the percentage of images labeled correct).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

import tensorflow as tf # Deep Learning Yo!

Loading **taining** and **testing** data

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

Seperating features (images : X) and labels (correct digit labels: y)

In [ ]:
y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1); del train

## 1. Data exploration

Checking the count of each label

In [ ]:
plt.figure(figsize=(12, 4))
sns.countplot(y_train)

In [ ]:
y_train.value_counts()

We can see that there are more or less similar number of images for each digit. So its less likely that our classifier will get biased due to representation error (eg: suppose the digit 5 being predicted most of the time because there are a larger number of images of 5s in our dataset compared to other numbers).

## 2. Data preparation

### 2.1 Reshaping images
The images are currently flattened: 28x28 grayscale images flattened = 784 pixels. This is so that it is easier to pass into traditional Machine Learning classifiers such as SVCs or Random Forest classifiers. However, for image data Convolutional Neural Networks (CNNs) are currently the best choice. Therefore, we will reshape the images into their original dimensions: 28x28x1 (1 channel).

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1) # Training images
test = test.values.reshape(-1, 28, 28, 1)       # Test set

Checking a few random images:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(8, 8))
for i in range(5):
    index = np.random.randint(0, len(X_train)-1)
    axes[i].imshow(X_train[index][:, :, 0], cmap='binary')
    print(y_train[index], end="\t\t")
fig.tight_layout()

### 2.2 Normalization
We will normalize the images to reduce the impact of alpha differences. Normalization also helps models converge faster.

In [ ]:
X_train = X_train / 255.0
test = test / 255.0

Checking impact of normalization on images:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(8, 8))
for i in range(5):
    index = np.random.randint(0, len(X_train)-1)
    axes[i].imshow(X_train[index][:, :, 0], cmap='binary')
    print(y_train[index], end="\t\t")
fig.tight_layout()

Normalization doesn't fundamentally change our image. It only shifts the pixel values so that they have mean of 0 and standard deviation of 1.

### 2.3 One-hot encoding for labels
One-hot encoding is a process by which categorical variables like $[cat, dog]$ or $[0, 1, 2,...,9]$ are converted into one-hot vectors. 

In this case, we have 10 classes - the numbers 0-9. Now, suppose a random image label $ y_i = 5 $, one-hot encoding will represent this as a one-hot vector $[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]$, similarly if $ y_i = 0 $, it will be represented as $[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]$.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')

Y_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1)).toarray()

Original label, $ y_0 $:

In [ ]:
y_train[0]

After one-hot encoding, $Y_0$:

In [ ]:
Y_train[0]

Image, $X_0$:

In [ ]:
plt.imshow(X_train[0][:, :, 0], cmap='binary')

We can see that the labels are now represented differently. Instead of a single integer, it is now represented by a one-hot vector.

### 2.4 Seperating training data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [ ]:
len(X_train), len(X_val), len(Y_train), len(Y_val)

## 3. Modeling
For implementing Convolutional Networks, we will use Tensorflow's official high level API - Keras.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 3.1 Building the model

In [ ]:
model = Sequential() # Instance of the Sequential model class

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28,28,1))) # Input
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu')) # CONV hidden layer 1
model.add(MaxPool2D(pool_size=(2,2))) # Pooling 1
model.add(Dropout(0.25)) # Regularization

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu')) # CONV hidden layer 2
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu')) # CONV hidden layer 3
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2))) # Pooling 2
model.add(Dropout(0.25)) # Regularization

model.add(Flatten()) # 2D -> 1D
model.add(Dense(256, activation="relu")) # FC (fully connected) hidden layer
model.add(Dropout(0.5)) # Regularization

model.add(Dense(10, activation="softmax")) # Output prediction

In [ ]:
# Defining optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
model.fit(X_train, Y_train, batch_size=86, epochs=3, validation_data=(X_val, Y_val), verbose=2)